In [110]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scipy

In [111]:
import pandas as pd 
import numpy as np
from scipy.spatial.distance import cosine

In [112]:
ratings = pd.read_csv('../jiangjunyi/' + 'ratings2.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [113]:
movies = pd.read_csv('../jiangjunyi/' + 'movies2.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [114]:
OurData = pd.merge(ratings, movies,how = 'inner')
OurData

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [115]:
OurData.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [116]:
rating_count_by_movie = OurData.groupby(['movieId','title'],as_index=False)['rating'].count()
rating_count_by_movie.columns=['movieId','title','rating_count']
rating_count_by_movie.sort_values(by=['rating_count'],ascending=False,inplace=True)
rating_count_by_movie[:10]

,movieId,title,rating_count
314,356,Forrest Gump (1994),329
277,318,"Shawshank Redemption, The (1994)",317
257,296,Pulp Fiction (1994),307
510,593,"Silence of the Lambs, The (1991)",279
1938,2571,"Matrix, The (1999)",278
224,260,Star Wars: Episode IV - A New Hope (1977),251
418,480,Jurassic Park (1993),238
97,110,Braveheart (1995),237
507,589,Terminator 2: Judgment Day (1991),224
461,527,Schindler's List (1993),220


In [117]:
rating_stddev = OurData.groupby(['movieId','title']).agg({'rating':['mean','std']})
rating_stddev.head(10)

rating          
                                                mean       std
movieId title                                                 
1       Toy Story (1995)                    3.920930  0.834859
2       Jumanji (1995)                      3.431818  0.881713
3       Grumpier Old Men (1995)             3.259615  1.054823
4       Waiting to Exhale (1995)            2.357143  0.852168
5       Father of the Bride Part II (1995)  3.071429  0.907148
6       Heat (1995)                         3.946078  0.817224
7       Sabrina (1995)                      3.185185  0.977561
8       Tom and Huck (1995)                 2.875000  1.125992
9       Sudden Death (1995)                 3.125000  0.974679
10      GoldenEye (1995)                    3.496212  0.859381

In [118]:
user = OurData.userId.unique()
item = OurData.title.unique()

In [119]:
Matrix = pd.DataFrame(columns=item, index=user)
Matrix.head(5)

,Toy Story (1995),Grumpier Old Men (1995),Heat (1995),Seven (a.k.a. Se7en) (1995),"Usual Suspects, The (1995)",From Dusk Till Dawn (1996),Bottle Rocket (1996),Braveheart (1995),Rob Roy (1995),Canadian Bacon (1995),...,Return of the One-Armed Swordsman (1969),Hitchcock/Truffaut (2015),He Never Died (2015),Southbound (2016),Gen-X Cops (1999),Bloodmoon (1997),Sympathy for the Underdog (1971),Hazard (2005),Blair Witch (2016),31 (2016)
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
for index, row in OurData.iterrows():
    currentUser = row['userId']
    currentContent = row['title']
    Matrix.at[currentUser, currentContent] = row['rating']
Matrix

,Toy Story (1995),Grumpier Old Men (1995),Heat (1995),Seven (a.k.a. Se7en) (1995),"Usual Suspects, The (1995)",From Dusk Till Dawn (1996),Bottle Rocket (1996),Braveheart (1995),Rob Roy (1995),Canadian Bacon (1995),...,Return of the One-Armed Swordsman (1969),Hitchcock/Truffaut (2015),He Never Died (2015),Southbound (2016),Gen-X Cops (1999),Bloodmoon (1997),Sympathy for the Underdog (1971),Hazard (2005),Blair Witch (2016),31 (2016)
1,4,4,4,5,5,3,5,4,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4,NaN,NaN,NaN,4,NaN,NaN,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.5,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2.5,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,4.5,NaN,NaN,4,4.5,NaN,NaN,4.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
def userCF_prediction(df, currentUser, numUsers, numItems):
    df = df.astype(float)
    cuDf = df.loc[currentUser]
    corrDf = df.corrwith(cuDf, axis=1, method='pearson')
    corrDf.sort_values(ascending=False, inplace=True)
    corrDf.drop(labels = corrDf[corrDf.values == 1.0].index, inplace=True)
    corrDf = corrDf.head(numUsers)
    toPredict = cuDf.dropna()
    ratings = df.loc[corrDf.index]
    ratingsToPredict = ratings[toPredict.index]
    predictedRatings = ratingsToPredict.mean()
    predictedRatings.sort_values(ascending=False, inplace=True)
    return predictedRatings.head(numItems)

In [122]:
userCF_prediction(Matrix, 15, 5, 5)

D:\Tools\Anaconda\lib\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
D:\Tools\Anaconda\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


Forrest Gump (1994)                                       4.666667
(500) Days of Summer (2009)                               4.000000
Minority Report (2002)                                    4.000000
101 Dalmatians (One Hundred and One Dalmatians) (1961)    3.500000
Lord of the Rings: The Two Towers, The (2002)             3.000000
dtype: float64

In [123]:
Matrix['American History X (1998)'][1]

5.0

In [124]:
user2 = OurData.userId.unique()
item2 = OurData.movieId.unique()

In [118]:
links = pd.read_csv('../jiangjunyi/' + 'links2.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [125]:
Matrix2 = pd.DataFrame(columns=item2, index=user2)
for index, row in OurData.iterrows():
    currentUser = row['userId']
    currentContent = row['movieId']
    Matrix2.at[currentUser, currentContent] = row['rating']
Matrix2

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
1,4,4,4,5,5,3,5,4,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4,NaN,NaN,NaN,4,NaN,NaN,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.5,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2.5,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,4.5,NaN,NaN,4,4.5,NaN,NaN,4.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
Matrix2 = Matrix2.astype(float)

In [127]:
currentItem = 3

In [128]:
dfSelection = Matrix2.dropna(subset = [currentItem])

In [129]:
dfSelection

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
1,4.0,4.0,4.0,5.0,5.0,3.0,5.0,4.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,4.0,3.0,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,3.0,3.0,3.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,5.0,5.0,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,4.0,3.5,4.5,4.5,3.0,4.0,NaN,4.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,2.5,2.0,4.0,4.0,3.0,4.0,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,4.0,3.0,5.0,4.5,4.5,NaN,NaN,4.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,4.5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
tf7 = dfSelection.fillna(dfSelection.mean(1))
tf7

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
1,4.000000,4.0,4.000000,5.0,5.0,3.0,5.0,4.0,5.000000,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,4.000000,3.0,3.493631,3.0,NaN,2.0,NaN,NaN,3.542373,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,3.000000,3.0,3.000000,NaN,5.0,NaN,NaN,NaN,3.542373,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,5.000000,5.0,3.493631,4.0,NaN,NaN,NaN,3.0,3.542373,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,3.000000,3.0,3.000000,NaN,NaN,NaN,NaN,NaN,3.542373,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,4.000000,3.5,4.500000,4.5,3.0,4.0,NaN,4.0,3.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,2.500000,2.0,4.000000,4.0,3.0,4.0,NaN,2.5,3.542373,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,4.000000,3.0,5.000000,4.5,4.5,NaN,NaN,4.0,3.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,5.000000,3.0,3.493631,NaN,NaN,NaN,NaN,NaN,3.542373,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,4.500000,5.0,3.493631,NaN,NaN,NaN,NaN,NaN,3.542373,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
dfSelection = dfSelection.dropna(axis = 1)
dfSelection

,3
1,4.0
19,3.0
32,3.0
43,5.0
44,3.0
64,3.5
68,2.0
91,3.0
151,3.0
169,5.0


In [131]:
currentUser = 1

In [133]:
dfSelection = Matrix2.dropna(subset = [currentItem])

In [134]:
cuDf = Matrix2.loc[currentUser]
moveDf = cuDf[cuDf.isna()]
dfSelection = dfSelection.drop(moveDf.index,axis = 1)
dfSelection

,1,3,6,47,50,70,101,110,151,157,...,3671,3702,3703,3729,3740,3744,3793,3809,4006,5060
1,4.0,4.0,4.0,5.0,5.0,3.0,5.0,4.0,5.0,5.0,...,5.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0
19,4.0,3.0,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,2.0,2.0,4.0,3.0,NaN,NaN
32,3.0,3.0,3.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,5.0,5.0,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,4.0,3.5,4.5,4.5,3.0,4.0,NaN,4.0,3.0,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,3.5,4.0,NaN,NaN
68,2.5,2.0,4.0,4.0,3.0,4.0,NaN,2.5,NaN,NaN,...,3.5,2.5,3.0,NaN,2.5,NaN,4.5,3.5,NaN,NaN
91,4.0,3.0,5.0,4.5,4.5,NaN,NaN,4.0,3.0,NaN,...,4.5,NaN,5.0,NaN,5.0,NaN,4.0,4.0,NaN,2.0
151,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,4.5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN


In [135]:
tf = dfSelection.copy()
on = tf.mean(axis = 1)

In [136]:
on

1      4.366379
19     3.040650
32     3.954545
43     4.409091
44     3.384615
64     3.828829
68     3.165584
91     3.764957
151    3.800000
169    4.195122
179    3.950000
217    3.016667
226    3.841837
240    4.080000
269    4.142857
270    3.900000
288    3.435065
307    3.210526
330    3.703704
337    4.571429
414    3.885000
448    3.676259
456    4.111111
470    3.727273
477    3.785714
480    3.719231
492    4.142857
544    4.166667
555    4.060606
590    3.582031
599    3.313889
608    3.165441
6      3.939394
42     3.860759
51     3.712500
58     4.153846
100    3.844828
102    3.941176
116    3.333333
117    3.484848
150    3.714286
289    3.611111
294    3.095238
302    4.250000
308    1.833333
321    3.714286
368    3.205607
410    3.480000
501    3.666667
552    3.407895
588    3.619048
594    4.285714
dtype: float64

In [137]:
for item in tf:
    for indexs in tf[item].isnull().index:
        if tf[item].isnull()[indexs] == True:
            tf[item][indexs] = on[indexs]
tf

,1,3,6,47,50,70,101,110,151,157,...,3671,3702,3703,3729,3740,3744,3793,3809,4006,5060
1,4.000000,4.0,4.000000,5.000000,5.000000,3.000000,5.000000,4.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,5.000000
19,4.000000,3.0,3.040650,3.000000,3.040650,2.000000,3.040650,3.040650,3.040650,3.040650,...,3.000000,3.040650,3.040650,3.040650,2.000000,2.000000,4.000000,3.000000,3.040650,3.040650
32,3.000000,3.0,3.000000,3.954545,5.000000,3.954545,3.954545,3.954545,3.954545,3.954545,...,3.954545,3.954545,3.954545,3.954545,3.954545,3.954545,3.954545,3.954545,3.954545,3.954545
43,5.000000,5.0,4.409091,4.000000,4.409091,4.409091,4.409091,3.000000,4.409091,4.409091,...,4.409091,4.409091,4.409091,4.409091,4.409091,4.409091,4.409091,4.409091,4.409091,4.409091
44,3.000000,3.0,3.000000,3.384615,3.384615,3.384615,3.384615,3.384615,3.384615,3.384615,...,3.384615,3.384615,3.384615,3.384615,3.384615,3.384615,3.384615,3.384615,3.384615,3.384615
64,4.000000,3.5,4.500000,4.500000,3.000000,4.000000,3.828829,4.000000,3.000000,3.828829,...,3.500000,3.828829,3.828829,3.828829,3.828829,3.828829,3.500000,4.000000,3.828829,3.828829
68,2.500000,2.0,4.000000,4.000000,3.000000,4.000000,3.165584,2.500000,3.165584,3.165584,...,3.500000,2.500000,3.000000,3.165584,2.500000,3.165584,4.500000,3.500000,3.165584,3.165584
91,4.000000,3.0,5.000000,4.500000,4.500000,3.764957,3.764957,4.000000,3.000000,3.764957,...,4.500000,3.764957,5.000000,3.764957,5.000000,3.764957,4.000000,4.000000,3.764957,2.000000
151,5.000000,3.0,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,...,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000,3.800000
169,4.500000,5.0,4.195122,4.195122,4.195122,4.195122,4.195122,4.195122,4.195122,4.195122,...,4.195122,4.195122,4.195122,4.195122,4.195122,4.195122,4.195122,4.000000,4.195122,4.195122


In [287]:
def itemCF_prediction2(df, currentUser, currentItem, numItems):
    df = df.astype(float)
    dfSelection = df.dropna(subset = [currentItem]) 
    cuDf = df.loc[currentUser]
    moveDf = cuDf[cuDf.isna()]
    dfSelection = dfSelection.drop(moveDf.index,axis = 1)
    on = dfSelection.mean(axis = 1)
    for item in dfSelection:
        for indexs in dfSelection[item].isnull().index:
            if dfSelection[item].isnull()[indexs] == True:
                dfSelection[item][indexs] = on[indexs]
    dfSelection = dfSelection.drop(index = currentUser)
    cuAvgRating = df.loc[[currentUser]].dropna(axis=1).mean(axis=1)
    ouAvgRating = dfSelection.mean(axis=1)
    ciRatings = dfSelection.pop(currentItem)
    dfSelection = dfSelection.sub(dfSelection.mean(axis=1), axis=0)
 
    def cosine_sim(df1, df2):
        df1na = df1.isna()
        df1clean = df1[~df1na]
        df2clean = df2[~df1na]

        df2na = df2clean.isna()
        df1clean = df1clean[~df2na]
        df2clean = df2clean[~df2na]

        distance = cosine(df1clean, df2clean)
        sim = 1 - distance
    
        return sim

    iiSimilarity = dfSelection.apply(lambda x: cosine_sim(ciRatings, x), axis=0)
    iiSimilarity.sort_values(ascending=False, inplace=True)
    itemsToCompare = dfSelection[iiSimilarity.head(numItems).index]
    predictedRatings = itemsToCompare.mean(axis=0)
    predictedRatings.sort_values(ascending=False, inplace=True)
    predictedRatings += cuAvgRating.iloc[0]
    
    byt = []
    for index in predictedRatings.index:
        col = OurData['title'][OurData['movieId'] == index].unique()
        byt.append(col[0])
    predictedRatings.index = byt
    
    return predictedRatings

In [288]:
tf5 = itemCF_prediction2(Matrix2,1,1,5)
tf5

Star Wars: Episode IV - A New Hope (1977)                                         4.696767
Star Wars: Episode V - The Empire Strikes Back (1980)                             4.665047
Schindler's List (1993)                                                           4.633331
Silence of the Lambs, The (1991)                                                  4.618850
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    4.601365
dtype: float64

In [289]:
AveragePrecision(tf5, Matrix ,1)

0.95

In [305]:
tf6 = itemCF_prediction2(Matrix2,1,1,5)
tf6

Star Wars: Episode IV - A New Hope (1977)                                         4.696767
Star Wars: Episode V - The Empire Strikes Back (1980)                             4.665047
Schindler's List (1993)                                                           4.633331
Forrest Gump (1994)                                                               4.626720
Silence of the Lambs, The (1991)                                                  4.618850
Matrix, The (1999)                                                                4.617924
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    4.601365
Monty Python and the Holy Grail (1975)                                            4.600378
Princess Bride, The (1987)                                                        4.571063
Welcome to Woop-Woop (1997)                                                       4.367024
dtype: float64

In [306]:
AveragePrecision(tf6, Matrix ,1)

1.0

In [329]:
tf7 = itemCF_prediction2(Matrix2,32,50,5)
tf7

Pulp Fiction (1994)                 4.214805
Shawshank Redemption, The (1994)    4.178750
Schindler's List (1993)             4.068101
Silence of the Lambs, The (1991)    4.035676
Touch (1997)                        3.758649
dtype: float64

In [308]:
Matrix.iloc[19]

Toy Story (1995)                      5
Grumpier Old Men (1995)             NaN
Heat (1995)                           3
Seven (a.k.a. Se7en) (1995)         NaN
Usual Suspects, The (1995)            5
                                   ... 
Bloodmoon (1997)                    NaN
Sympathy for the Underdog (1971)    NaN
Hazard (2005)                       NaN
Blair Witch (2016)                  NaN
31 (2016)                           NaN
Name: 57, Length: 9719, dtype: object

In [330]:
AveragePrecision(tf7, Matrix ,32)

0.9166666666666666

In [302]:
RMSE(tf7, Matrix, 1)

RMSE: 1.1512272634999532


In [ ]:
SUM = 0
count = 0
for index in Matrix:
    count += 1
    Pred = itemCF_prediction2(Matrix, index , 9 , 5)
    AP = AveragePrecision(Pred, Matrix,index)
    SUM += AP
MAP = SUM/count

In [192]:
Matrix['Welcome to Woop-Woop (1997)'][1]

4.0

In [200]:
def RMSE(testX, testY ,CurrentUser):
    count = 0
    sm = 0
    for index in testX.index:
        if testY[index][CurrentUser] is not np.nan:
            sm += (testX[index] - testY[index][CurrentUser])*(testX[index] - testY[index][CurrentUser])
            count += 1
        else:
            sm += (testX[index] - testY[index].mean())*(testX[index] -testY[index].mean())
            count += 1
    Result = (sm/count)**0.5
    print('RMSE:', Result)

In [88]:
RMSE(tf,Matrix,5)

RMSE: 0.7444588377630147


In [89]:
RMSE(df5,Matrix,7)

RMSE: 3.1464079728926313


In [ ]:
itemsToCompare = dfSelection[iiSimilarity.head(numItems).index

In [200]:
Matrix['Usual Suspects, The (1995)'][17]

4.5

In [ ]:
abs(testY[index][CurrentUser]-testX[index])<0.5

In [316]:
def AveragePrecision(testX, testY, CurrentUser):
    count = 0
    score = 0
    location_score = 0
    hit = 0
    for index in testX.index:
        location_score += 1
        if testY[index][CurrentUser] >= 4.5:
            count += 1
            score += count/location_score
            hit += 1
    if hit ==0:
        final_score = 0
    else:
        final_score = score/hit
    return final_score       

In [109]:
tf = userCF_prediction(Matrix, 17, 7, 5)
tf

Series([], dtype: float64)

In [229]:
tf = userCF_prediction(Matrix, 17, 7, 5)
tf

0.41666666666666663

In [232]:
tf2 = userCF_prediction(Matrix, 17, 8, 5)
AveragePrecision(tf2, Matrix , 17)

0.41666666666666663

In [234]:
tf2 = userCF_prediction(Matrix, 17, 9, 5)
AveragePrecision(tf2, Matrix , 17)

1.0

In [267]:
tf3 = userCF_prediction(Matrix, 5, 7, 5)
tf3

Schindler's List (1993)           5.0
Toy Story (1995)                  4.5
Remains of the Day, The (1993)    4.0
Aladdin (1992)                    4.0
Pulp Fiction (1994)               4.0
dtype: float64

In [268]:
AveragePrecision(tf3, Matrix ,5)

0.75

In [246]:
tf4 = userCF_prediction(Matrix, 15, 8, 5)
tf4

Independence Day (a.k.a. ID4) (1996)    5.00
Forrest Gump (1994)                     4.75
Pulp Fiction (1994)                     4.50
(500) Days of Summer (2009)             4.00
Minority Report (2002)                  4.00
dtype: float64

In [247]:
AveragePrecision(tf4, Matrix , 15)

0.325

In [271]:
tf5 = userCF_prediction(Matrix, 19, 9, 5)
tf5

Raising Arizona (1987)                                                            5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Back to the Future (1985)                                                         5.0
Blade Runner (1982)                                                               5.0
American Beauty (1999)                                                            5.0
dtype: float64

In [273]:
AveragePrecision(tf5, Matrix , 19)

0.5

In [254]:
SUM = 0
count = 0
for index in Matrix:
    count += 1
    Pred = userCF_prediction(Matrix, index , 9 , 5)
    AP = AveragePrecision(Pred, Matrix,index)
    SUM += AP
MAP = SUM/count

KeyError: 'Toy Story (1995)'